![image](https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/notebooks/headers/AutoAI-Banner_Experiment-Notebook.png)
# Experiment Notebook - AutoAI Notebook v1.15.3


This notebook contains the steps and code to demonstrate support of AutoAI experiments in Watson Machine Learning service. It introduces Python API commands for data retrieval, training experiments, persisting pipelines, testing pipelines, refining pipelines, and scoring the resulting model.

**Note:** Notebook code generated using AutoAI will execute successfully. If code is modified or reordered, there is no guarantee it will successfully execute. For details, see: <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-notebook.html">Saving an Auto AI experiment as a notebook</a>


Some familiarity with Python is helpful. This notebook uses Python 3.8 and `ibm_watson_machine_learning` package.


## Notebook goals

The learning goals of this notebook are:
-  Defining an AutoAI experiment
-  Training AutoAI models 
-  Comparing trained models
-  Deploying the model as a web service
-  Scoring the model to generate predictions.



## Contents

This notebook contains the following parts:

**[Setup](#setup)**<br>
&nbsp;&nbsp;[Package installation](#install)<br>
&nbsp;&nbsp;[Watson Machine Learning connection](#connection)<br>
**[Experiment configuration](#configuration)**<br>
&nbsp;&nbsp;[Experiment metadata](#metadata)<br>
**[Working with completed AutoAI experiment](#work)**<br>
&nbsp;&nbsp;[Get fitted AutoAI optimizer](#get)<br>
&nbsp;&nbsp;[Pipelines comparison](#comparison)<br>
&nbsp;&nbsp;[Get pipeline as scikit-learn pipeline model](#get_pipeline)<br>
&nbsp;&nbsp;[Inspect pipeline](#inspect_pipeline)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[Visualize pipeline model](#visualize)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[Preview pipeline model as python code](#preview)<br>
**[Deploy and Score](#scoring)**<br>
&nbsp;&nbsp;[Working with spaces](#working_spaces)<br>
**[Running AutoAI experiment with Python API](#run)**<br>
**[Clean up](#cleanup)**<br>
**[Next steps](#next_steps)**<br>
**[Copyrights](#copyrights)**

<a id="setup"></a>
# Setup

<a id="install"></a>
## Package installation
Before you use the sample code in this notebook, install the following packages:
 - ibm-watson-machine-learning,
 - autoai-libs,
 - lale,
 - scikit-learn,
 - xgboost,
 - lightgbm,
 - snapml.


In [1]:
!pip install ibm-watson-machine-learning | tail -n 1
!pip install -U autoai-libs==1.12.13 | tail -n 1
!pip install -U 'lale>=0.5.3,<0.6' | tail -n 1
!pip install -U scikit-learn==0.23.2 | tail -n 1
!pip install -U xgboost==1.3.3 | tail -n 1
!pip install -U 'lightgbm>=3.1.1,<=3.2.1' | tail -n 1
!pip install -U 'snapml>=1.7.4,<1.8.0' | tail -n 1

<a id="configuration"></a>
# Experiment configuration

<a id="metadata"></a>
## Experiment metadata
This cell defines the metadata for the experiment, including: training_data_reference, training_result_reference, experiment_metadata.

In [2]:
from ibm_watson_machine_learning.helpers import DataConnection
from ibm_watson_machine_learning.helpers import FSLocation

training_data_reference = [
    DataConnection(
        data_asset_id='4ea5abf4-f695-46d4-888c-6db25d5a7b2f'
    ),
]
training_result_reference = DataConnection(
    location=FSLocation(
        path='/projects/44b8751c-098f-484b-8c66-ebf128cdd731/assets/auto_ml/bd8a8621-10e2-46f0-ad5a-3ffefafddf24/wml_data/b955d67e-d129-4931-9d8d-706442c24d98/data/automl'
    )
)

In [7]:
# Import the lib
from project_lib import Project
project = Project.access()

2022-02-09 21:11:46,471 - __PROJECT_LIB__ - ERROR - create_storage_adapter: unknown storage type 'local_git_storage'


RuntimeError: Unknown storage type in metadata.

In [6]:
project_id

NameError: name 'project_id' is not defined

In [3]:
experiment_metadata = dict(
    prediction_type='binary',
    prediction_column='pred_FIRST',
    holdout_size=0.15,
    scoring='roc_auc',
    csv_separator=',',
    random_state=33,
    max_number_of_estimators=2,
    training_data_reference=training_data_reference,
    training_result_reference=training_result_reference,
    include_only_estimators=['LogisticRegression', 'RandomForestClassifier', 'DecisionTreeClassifier', 'GradientBoostingClassifier'],
    project_id='44b8751c-098f-484b-8c66-ebf128cdd731',
    train_sample_columns_index_list=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    positive_label='Y',
    drop_duplicates=True
)

<a id="connection"></a>
## Watson Machine Learning connection

This cell defines the credentials required to work with the Watson Machine Learning service.

**Action**: You need to pass yours Cloud Pak for Data instance home url.   
**Note**: If you want to run the notebook outside of the Watson Studio please provide Watson Machine Learning credentials: [Documentation](http://ibm-wml-api-pyclient.mybluemix.net/#id6)
```
wml_credentials = {  
    "instance_id": "openshift",  
    "token": "",  
    "url": "",  
    "version": "4.0"  
}
```

In [4]:
import os

wml_credentials = {
    "instance_id": "openshift",
    "token": os.environ.get("USER_ACCESS_TOKEN"),
    "url": os.environ.get("RUNTIME_ENV_APSX_URL"),
    "version": "4.0"
}

<a id="work"></a>


# Working with the completed AutoAI experiment

This cell imports the pipelines generated for the experiment so they can be compared to find the optimal pipeline to save as a model.

<a id="get"></a>


## Get fitted AutoAI optimizer

In [5]:
from ibm_watson_machine_learning.experiment import AutoAI

pipeline_optimizer = AutoAI(wml_credentials, project_id=experiment_metadata['project_id']).runs.get_optimizer(metadata=experiment_metadata)

Cannot set Project or Space
Reason: {"code":404,"error":"Not Found","reason":"Failed to retrieve project: 44b8751c-098f-484b-8c66-ebf128cdd731. Failed to retrieve Cloudant document: 44b8751c-098f-484b-8c66-ebf128cdd731. missing.","message":"Resource requested by the client was not found."}


CannotSetProjectOrSpace: Cannot set Project or Space
Reason: {"code":404,"error":"Not Found","reason":"Failed to retrieve project: 44b8751c-098f-484b-8c66-ebf128cdd731. Failed to retrieve Cloudant document: 44b8751c-098f-484b-8c66-ebf128cdd731. missing.","message":"Resource requested by the client was not found."}

Use `get_params()`- to retrieve configuration parameters.

In [ ]:
pipeline_optimizer.get_params()

<a id="comparison"></a>
## Pipelines comparison

Use the `summary()` method to list trained pipelines and evaluation metrics information in
the form of a Pandas DataFrame. You can use the DataFrame to compare all discovered pipelines and select the one you like for further testing.

In [ ]:
summary = pipeline_optimizer.summary()
best_pipeline_name = list(summary.index)[0]
summary

<a id="get_pipeline"></a>
### Get pipeline as scikit-learn pipeline model

After you compare the pipelines, download and save a scikit-learn pipeline model object from the
AutoAI training job.

**Tip:** To get a specific pipeline pass the pipeline name in:
```
pipeline_optimizer.get_pipeline(pipeline_name=pipeline_name)
```

In [ ]:
pipeline_model = pipeline_optimizer.get_pipeline()

Next, check features importance for selected pipeline.

In [ ]:
pipeline_optimizer.get_pipeline_details()['features_importance']

**Tip:** If you want to check all model evaluation metrics-details, use:
```
pipeline_optimizer.get_pipeline_details()
```

<a id="inspect_pipeline"></a>
## Inspect pipeline

<a id="visualize"></a>
### Visualize pipeline model

Preview pipeline model stages as a graph. Each node's name links to a detailed description of the stage.


In [ ]:
pipeline_model.visualize()

<a id="preview"></a>
### Preview pipeline model as Python code
In the next cell, you can preview the saved pipeline model as Python code.  
You can review the exact steps used to create the model.

**Note:** If you want to get sklearn representation, add the following parameter to `pretty_print` call: `astype='sklearn'`.

In [ ]:
pipeline_model.pretty_print(combinators=False, ipython_display=True)

### Calling the `predict` method
If you want to get a prediction using pipeline model object, call `pipeline_model.predict()`.

**Note:** If you want to work with pure sklearn model:
 - add the following parameter to `get_pipeline` call: `astype='sklearn'`,
 - or `scikit_learn_pipeline = pipeline_model.export_to_sklearn_pipeline()`

<a id="scoring"></a>
## Deploy and Score

In this section you will learn how to deploy and score the model as a web service.

<a id="working_spaces"></a>
### Working with spaces

In this section you will specify a deployment space for organizing the assets for deploying and scoring the model. If you do not have an existing space, go to **Deployments** section and create a new space, following these steps:

- Click **New Deployment Space**.
- Create an empty space.
- View new space.
- Go to **Manage** bookmark.
- Copy `space_id` and paste it below.

**Tip**: You can also use the API to prepare the space for your work. Learn more [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd4.0/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: assign or update space ID below.

### Deployment creation

In [ ]:
target_space_id = "PUT_YOUR_TARGET_SPACE_ID_HERE"

from ibm_watson_machine_learning.deployment import WebService

service = WebService(
    source_wml_credentials=wml_credentials,
    target_wml_credentials=wml_credentials,
    source_project_id=experiment_metadata['project_id'],
    target_space_id=target_space_id
)
service.create(
    model=best_pipeline_name,
    metadata=experiment_metadata,
    deployment_name='Best_pipeline_webservice'
)

Use the `print` method for the deployment object to show basic information about the service: 

In [ ]:
print(service)

To show all available information about the deployment use the `.get_params()` method.

In [ ]:
service.get_params()

### Scoring of webservice
You can make scoring request by calling `score()` on the deployed pipeline.

If you want to work with the web service in an external Python application, follow these steps to retrieve the service object:

 - Initialize the service by `service = WebService(target_wml_credentials=wml_credentials,target_space_id=experiment_metadata['space_id'])`
 - Get deployment_id by `service.list()` method
 - Get webservice object by `service.get('deployment_id')` method

After that you can call `service.score(score_records_df)` method. The `score()` method accepts `pandas.DataFrame` object. 

<a id="cleanup"></a>
### Deleting deployment
You can delete the existing deployment by calling the `service.delete()` command.
To list the existing web services, use `service.list()`.

<a id="run"></a>

## Running AutoAI experiment with Python API

If you want to run the AutoAI experiment using the Python API, follow these. The experiment settings were generated basing on parameters set in the AutoAI UI.


```
from ibm_watson_machine_learning.experiment import AutoAI

experiment = AutoAI(wml_credentials, project_id=experiment_metadata['project_id'])
```

```
OPTIMIZER_NAME = 'custom_name'
```

```
from ibm_watson_machine_learning.helpers import DataConnection
from ibm_watson_machine_learning.helpers import FSLocation

training_data_reference = [
    DataConnection(
        data_asset_id='4ea5abf4-f695-46d4-888c-6db25d5a7b2f'
    ),
]
training_result_reference = DataConnection(
    location=FSLocation(
        path='/projects/44b8751c-098f-484b-8c66-ebf128cdd731/assets/auto_ml/bd8a8621-10e2-46f0-ad5a-3ffefafddf24/wml_data/b955d67e-d129-4931-9d8d-706442c24d98/data/automl'
    )
)
```

The new pipeline optimizer will be created and training will be triggered.

```
pipeline_optimizer = experiment.optimizer(
    name=OPTIMIZER_NAME,
    prediction_type=experiment_metadata['prediction_type'],
    prediction_column=experiment_metadata['prediction_column'],
    scoring=experiment_metadata['scoring'],
    holdout_size=experiment_metadata['holdout_size'],
    csv_separator=experiment_metadata['csv_separator'],
    positive_label=experiment_metadata['positive_label'],
    drop_duplicates=experiment_metadata['drop_duplicates'],
    train_sample_columns_index_list=experiment_metadata['train_sample_columns_index_list'],
    include_only_estimators=experiment_metadata['include_only_estimators']
)
```

```
pipeline_optimizer.fit(
    training_data_reference=training_data_reference,
    training_results_reference=training_result_reference,
    background_mode=False,
)
```


<a id="next_steps"></a>
# Next steps

#### [Online Documentation](https://www.ibm.com/support/producthub/icpdata/docs/content/SSQNUZ_latest/cpd/overview/relnotes-4.0.html)

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2021 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for Watson Studio Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="http://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF">License Terms</a>  

___